In [1]:
import pandas as pd
import os
import gzip
import json
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import requests
import datetime
from scipy import stats
from load_helper_CoronaWiki import *
import urllib.parse
import time 
COVID_RELATED_ARTICLES_PATH = "data/COVID_related_pages_project-1.csv"
DICT_COUNTRIES_OWN_LANG = {"Italy" : "it", "Russia": "ru", "China": "zh", "India": "hi", "Albania": "sq", 
"Bangladesh": "bn", "Bostwana": "tn", "Cambogia": "khm", "Croatia": "hr", "Greece": "el", "Sweden": "sv", "Finland": "fi", "Norway": "no", "Malaysian": "ms", "Israel": "he",  }

In [2]:
df_covid_articles = pd.read_csv("data/COVID_related_pages_project-1.csv")

In [3]:
df_covid_articles.loc[df_covid_articles.project == "vi.wikipedia"]

,page,project,url,wikilink
17,Đại dịch COVID-19 tại Uruguay,vi.wikipedia,https://vi.wikipedia.org/wiki/%C4%90%E1%BA%A1i...,[[vi:Đại dịch COVID-19 tại Uruguay
37,Đại dịch COVID-19 tại Andorra,vi.wikipedia,https://vi.wikipedia.org/wiki/%C4%90%E1%BA%A1i...,[[vi:Đại dịch COVID-19 tại Andorra
73,Dòng thời gian của đại dịch COVID-19 vào tháng...,vi.wikipedia,https://vi.wikipedia.org/wiki/D%C3%B2ng_th%E1%...,[[vi:Dòng thời gian của đại dịch COVID-19 vào ...
84,Thể loại:Đại dịch COVID-19 tại châu Phi,vi.wikipedia,https://vi.wikipedia.org/wiki/Th%E1%BB%83_lo%E...,[[vi:Thể loại:Đại dịch COVID-19 tại châu Phi
96,Đại dịch COVID-19 tại Hồ Bắc,vi.wikipedia,https://vi.wikipedia.org/wiki/%C4%90%E1%BA%A1i...,[[vi:Đại dịch COVID-19 tại Hồ Bắc
...,...,...,...,...
5120,Đại dịch COVID-19 tại Hy Lạp,vi.wikipedia,https://vi.wikipedia.org/wiki/%C4%90%E1%BA%A1i...,[[vi:Đại dịch COVID-19 tại Hy Lạp
5135,Đại dịch COVID-19 tại Đài Loan,vi.wikipedia,https://vi.wikipedia.org/wiki/%C4%90%E1%BA%A1i...,[[vi:Đại dịch COVID-19 tại Đài Loan
5159,Đại dịch COVID-19 tại Bỉ,vi.wikipedia,https://vi.wikipedia.org/wiki/%C4%90%E1%BA%A1i...,[[vi:Đại dịch COVID-19 tại Bỉ
5173,Đại dịch COVID-19 tại Gabon,vi.wikipedia,https://vi.wikipedia.org/wiki/%C4%90%E1%BA%A1i...,[[vi:Đại dịch COVID-19 tại Gabon


In [8]:
df2 = df_covid_articles.groupby(['project']).aggregate('count')
df2 = df2.loc[df2.page > 10]
df2.shape

(60, 3)

In [5]:
'''
Function to be used within data fetching script
Inputs : 
    - json_obj : dict that was fetched in the given iteration of the script
    - return_df : final df that will be return at the end of the script
Output : concatenated df
'''
def json_to_df(json_obj, return_df):
    #Create df from json with nested list
    df = pd.json_normalize(json_obj, record_path = ['items']).set_index(['timestamp', 'article']).drop(labels = ['project', 'granularity'
    , 'access', 'agent'], axis = 1)
    #concatenation
    return  pd.concat([return_df, df])


In [6]:

def wiki_to_df_extract(languageCode, begin_date, end_date):
    df_covid_articles = pd.read_csv(COVID_RELATED_ARTICLES_PATH)
    df_covid_articles_country = df_covid_articles.loc[df_covid_articles.project == "{}.wikipedia".format(languageCode)]['page']
    df_agg_country = pd.DataFrame()
    print(len(df_covid_articles_country))
    for page in df_covid_articles_country:
        #time.sleep(1)
        page = page.replace(' ', '_')
        page = urllib.parse.quote(page)
        url = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/{}.wikipedia.org/all-access/user/{}/daily/{}/{}'.format(languageCode,page, begin_date, end_date)
        header = {'User-Agent' : 'Robin Debalme (academic project; robin.debalme@epfl.ch; https://github.com/epfl-ada/ada-2022-project-thedatadiggers22)'}
        try:
            r = requests.get(url, headers = header).json()
        except:
            print("Bad request")
        df_agg_country = json_to_df(r, df_agg_country)
    return df_agg_country.groupby(['timestamp'])['views'].sum().to_frame()

In [7]:

df_test = wiki_to_df_extract('fr', '20200101', '20210101')
print(df_test.head())

189
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est là wllh
On est

In [10]:
print(df_test.head(90))

            views
timestamp        
2020010100    242
2020010200    308
2020010300    384
2020010400    424
2020010500    445
...           ...
2020032600  43203
2020032700  41451
2020032800  40678
2020032900  43459
2020033000  43388

[90 rows x 1 columns]
